In [1]:
import pandas as pd

### Load Dataset

In [2]:
train = pd.read_csv("input/train.tsv", sep="\t", index_col="PhraseId")

print(train.shape)
train.head()

(156060, 3)


,SentenceId,Phrase,Sentiment
PhraseId,,,
1,1,A series of escapades demonstrating the adage ...,1
2,1,A series of escapades demonstrating the adage ...,2
3,1,A series,2
4,1,A,2
5,1,series,2


In [43]:
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from tqdm import tqdm

def pos_phrase(phrase):
    words = word_tokenize(phrase)
    word_pos = [word + "_" + pos for word, pos in pos_tag(words)]

    return " ".join(word_pos)

tqdm.pandas(desc="pos tagging...")
train["Phrase"].progress_apply(pos_phrase)

In [60]:
def find_wordnet_pos(pos):
    if "V" in pos:
        return "v"
    else:
        return "n"
    
print(find_wordnet_pos("VB"))
print(find_wordnet_pos("NN"))
print(find_wordnet_pos("RB"))

v
n
n


In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_phrase(phrase):
    words = word_tokenize(phrase)
    words_pos = pos_tag(words)

    lemmatized_words = []

    for word, pos in words_pos:
        lemmatized_word = lemmatizer.lemmatize(word, pos=find_wordnet_pos(pos))
        lemmatized_words.append(lemmatized_word)
        
    return " ".join(lemmatized_words)

tqdm.pandas(desc="pos tagging + lemmatizing...")
train["Phrase"].progress_apply(lemmatize_phrase)


pos tagging + lemmatizing...:  13%|█▎        | 19724/156060 [00:19<02:24, 943.61it/s]


pos tagging + lemmatizing...:  25%|██▌       | 39571/156060 [00:39<02:00, 970.25it/s]


pos tagging + lemmatizing...:  38%|███▊      | 60058/156060 [00:58<01:26, 1112.99it/s]


pos tagging + lemmatizing...:  51%|█████▏    | 80351/156060 [01:18<01:12, 1041.12it/s]


pos tagging + lemmatizing...:  64%|██████▍   | 99774/156060 [01:37<00:49, 1137.45it/s]


pos tagging + lemmatizing...:  77%|███████▋  | 120762/156060 [01:56<00:35, 995.36it/s]


pos tagging + lemmatizing...:  88%|████████▊ | 137638/156060 [02:16<00:22, 808.57it/s]


pos tagging + lemmatizing...:  93%|█████████▎| 144762/156060 [02:28<00:19, 579.09it/s]

In [3]:
test = pd.read_csv("input/test.tsv", sep="\t", index_col="PhraseId")

print(test.shape)
test.head()

(66292, 2)


,SentenceId,Phrase
PhraseId,,
156061,8545,An intermittently pleasing but mostly routine ...
156062,8545,An intermittently pleasing but mostly routine ...
156063,8545,An
156064,8545,intermittently pleasing but mostly routine effort
156065,8545,intermittently pleasing but mostly routine


## Preprocessing

In [4]:
train["Phrase(Origin)"] = train["Phrase"].copy()

print(train.shape)
train[["Phrase", "Phrase(Origin)"]].head()

(156060, 4)


,Phrase,Phrase(Origin)
PhraseId,,
1,A series of escapades demonstrating the adage ...,A series of escapades demonstrating the adage ...
2,A series of escapades demonstrating the adage ...,A series of escapades demonstrating the adage ...
3,A series,A series
4,A,A
5,series,series


In [5]:
test["Phrase(Origin)"] = test["Phrase"].copy()

print(test.shape)
test[["Phrase", "Phrase(Origin)"]].head()

(66292, 3)


,Phrase,Phrase(Origin)
PhraseId,,
156061,An intermittently pleasing but mostly routine ...,An intermittently pleasing but mostly routine ...
156062,An intermittently pleasing but mostly routine ...,An intermittently pleasing but mostly routine ...
156063,An,An
156064,intermittently pleasing but mostly routine effort,intermittently pleasing but mostly routine effort
156065,intermittently pleasing but mostly routine,intermittently pleasing but mostly routine


### Clean Text

In [6]:
def clean_text(phrase):
    phrase = phrase.replace("ca n't", "can not")
    phrase = phrase.replace("does n't", "does not")
    phrase = phrase.replace("n't", "not")

    return phrase

In [7]:
train["Phrase"] = train["Phrase"].apply(clean_text)

print(train.shape)
train[["Phrase", "Phrase(Origin)"]].head()

(156060, 4)


,Phrase,Phrase(Origin)
PhraseId,,
1,A series of escapades demonstrating the adage ...,A series of escapades demonstrating the adage ...
2,A series of escapades demonstrating the adage ...,A series of escapades demonstrating the adage ...
3,A series,A series
4,A,A
5,series,series


In [8]:
test["Phrase"] = test["Phrase"].apply(clean_text)

print(test.shape)
test[["Phrase", "Phrase(Origin)"]].head()

(66292, 3)


,Phrase,Phrase(Origin)
PhraseId,,
156061,An intermittently pleasing but mostly routine ...,An intermittently pleasing but mostly routine ...
156062,An intermittently pleasing but mostly routine ...,An intermittently pleasing but mostly routine ...
156063,An,An
156064,intermittently pleasing but mostly routine effort,intermittently pleasing but mostly routine effort
156065,intermittently pleasing but mostly routine,intermittently pleasing but mostly routine


### Vectorize Phrases

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 2))
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [10]:
vectorizer.fit(train["Phrase"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [11]:
X_train = vectorizer.transform(train["Phrase"])
X_train

<156060x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 1152072 stored elements in Compressed Sparse Row format>

In [12]:
vocabulary = vectorizer.get_feature_names()
pd.DataFrame(X_train[0:100].toarray(), columns=vocabulary).head()

,000,10,10 minutes,10 year,100,101,11,12,12 year,13,...,your watch,yourself,youth,youthful,zany,zeal,zero,zhang,zombie,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
X_test = vectorizer.transform(test["Phrase"])
X_test

<66292x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 421280 stored elements in Compressed Sparse Row format>

In [14]:
y_train = train["Sentiment"]

print(y_train.shape)
y_train.head()

(156060,)


PhraseId
1    1
2    2
3    2
4    2
5    2
Name: Sentiment, dtype: int64

### Train

In [15]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(random_state=37)
model

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=37, shuffle=True,
       tol=None, verbose=0, warm_start=False)

### Scoring

In [16]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GroupKFold

kfold = GroupKFold(n_splits=5)

y_predict = cross_val_predict(model, X_train, y_train,
                              cv=kfold, groups=train["SentenceId"])

print(y_predict.shape)
y_predict

/Users/shaynekang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/shaynekang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users

(156060,)


array([3, 3, 2, ..., 2, 2, 2])

In [17]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_train, y_predict)

print("Score = {0:0.6f}".format(score))

Score = 0.583602


In [18]:
import numpy as np

result = train.copy()
result["Sentiment(predict)"] = y_predict

result["Distance"] = result["Sentiment"] - result["Sentiment(predict)"]
result["Distance"] = np.abs(result["Distance"])

result = result.sort_values(by="Distance", ascending=False)
result.head()

,SentenceId,Phrase,Sentiment,Phrase(Origin),Sentiment(predict),Distance
PhraseId,,,,,,
30291,1407,"Like a skillful fisher , the director uses the...",4,"Like a skillful fisher , the director uses the...",0,4
41990,2021,To build a feel-good fantasy around a vain dic...,0,To build a feel-good fantasy around a vain dic...,4,4
82697,4270,This is one of the biggest disappointments of ...,0,This is one of the biggest disappointments of ...,4,4
82698,4270,is one of the biggest disappointments of the y...,0,is one of the biggest disappointments of the y...,4,4
82699,4270,is one of the biggest disappointments of the year,0,is one of the biggest disappointments of the year,4,4


In [19]:
# result[result["Phrase"].str.contains("can't")]

In [20]:
result[0:10000].to_csv("result.csv")

In [21]:
pd.DataFrame(vocabulary).to_csv("vocabulary.csv")

### Predict

In [22]:
model.fit(X_train, y_train)

/Users/shaynekang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=37, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [23]:
predictions = model.predict(X_test)

print(predictions.shape)
predictions

(66292,)


array([3, 3, 2, ..., 2, 2, 2])

## Submit

In [24]:
submission = pd.read_csv("input/sampleSubmission.csv", index_col="PhraseId")

print(submission.shape)
submission.head()

(66292, 1)


,Sentiment
PhraseId,
156061,2
156062,2
156063,2
156064,2
156065,2


In [25]:
submission["Sentiment"] = predictions

print(submission.shape)
submission.head()

(66292, 1)


,Sentiment
PhraseId,
156061,3
156062,3
156063,2
156064,3
156065,3


In [26]:
submission.to_csv("baseline-script.csv")